In [3]:
import pandas as pd 
import regex as re
from ingredient_parser import parse_ingredient
import random
from collections import Counter , defaultdict

In [4]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
 
# All dataframes hereafter reflect these changes.
# display(df)
# print('**RESET_OPTIONS**')
# Resets the options
# pd.reset_option('all')
# display(df)

In [5]:
def clean(row):
    row = re.sub(r"(\w)\n(\w)",r'\1 \2', row)
    row = re.sub(r"\s(\.|\,)",r'\1',  row)
    return row

def clean_ingr(row):
    row = re.sub(r"\n\s?",', ', row)
#     row = row.split(',')
    return row

def strip_ingr(row):
    row = re.sub('½|¹ / 2', '', row)
    row = re.sub(r'(\n)?\d/?\d*(?:ml|kg|g|l|\s)(?:tin|tbsp|tsp|packs|pack|bag|bags)?',r'\1', row)
    row = re.sub("\(.*\)",'', row)
    row = re.sub("\(.*\)",'', row)
    return row

# add to taste -\nadd to taste
# -? optional 
# - you 
# 1. 
# 2 x 1
# Ingr 
# 2-1 / 4

#وت تے

In [14]:
df = pd.read_csv('recipes.csv')
df.instructions = df.instructions.apply(clean)
df.ingredients = df.ingredients.apply(lambda x: re.sub(r'.*Ingredients\s?\n','',x))
df.ingredients = df.ingredients.apply(clean_ingr)
df['clean_ingr'] = df.ingredients.apply(strip_ingr)
df = df[~df.title.duplicated()]
# df['clean'] = df.ingredients.apply(lambda x: parse_ingredient(x).name.text.split(', '))

In [15]:
def parser(row): 
    obj = parse_ingredient(row)
    ingredients = obj.name.text.split(', ')
    whole = []
    units = []
    quantities = []
    ingrs = []
    for ing, a in zip(ingredients, obj.amount):
        temp = a.quantity + a.unit + " " + ing
#         whole.append(temp)
        units.append(a.unit)
        quantities.append(a.quantity)
        ingrs.append(ing)
    return units, quantities, ingrs, obj.comment.text
#     print(whole)
#     print(f'Additionally, these are some extras that might be essential and might be non-sense : {obj.comment.text}')

df['clean_ingredients'], df['units'],df['quantities'], df['addon'] = zip(*df.ingredients.apply(lambda x: parser(x)))

In [16]:
df.head()

,original_filename,title,instructions,ingredients,clean_ingr,clean_ingredients,units,quantities,addon
0,IMG_1271.JPG,Greek Mushroom Ragù & Olive Oil Mash,"1. Peel, halve and finely slice the onion. Pee...","1 red onion, 2 garlic cloves, 200g portobello ...","red onion, garlic cloves, portobello mushroom...","[, g, g, g, ml, ml, g, g, ]","[1, 200, 60, 500, 100, 200, 15, 200, 1]","[red onion, portobello mushrooms, green olives...","2 garlic cloves, 1 tin dark green lentils, 1 G..."
1,IMG_1270.JPG,Mushroom Risotto Stuffed Squash,1. Boil a kettle. Heat your oven to 190 ° C / ...,"1 leek, 10g rosemary, 5g dried mushrooms, 1 sp...","leek, rosemary, dried mushrooms, spaghetti s...","[, g, tbsp, g, g, head, g, tsps, tbsp]","[1, 10, 1, 200, 150, 1, 10, 0.25, 1]","[leek, rosemary, mushrooms, bouillon powder, b...","5 g dried, 1 spaghetti squash, 2 x 0.5 tbsp, 2..."
2,IMG_1268.JPG,Nasi Goreng with Peanuts & Sriracha,1. Boil a kettle.\n2. Using about a third of t...,"1/3 cucumber, 1 tbsp rice vinegar, 1 tbsp ligh...","cucumber, rice vinegar, light brown sugar, ...","[, tbsp, tbsp, g, bunch, bag, g, tsps, tsps, ,...","[0.333, 1, 1, 150, 1, 1, 30, 2, 0.25, 2, 2]","[cucumber, rice vinegar, light brown sugar, br...","1 pointed cabbage, 1 large or 2 smaller carrot..."
3,IMG_1269.JPG,Teriyaki Mushroom Rice & Asian Slaw,1. Boil a kettle. Rinse the rice in a sieve. T...,"150g brown basmati rice, 1 red onion, 1 carrot...","brown basmati rice, red onion, carrot, red ca...","[g, pack, tsps, g, g, g, ml, tbsp, tbsps, tbsp]","[150, 1, 0.25, 400, 15, 25, 50, 1, 2, 1]","[brown basmati rice, sweet chilli sauce, chill...","1 red onion, 1 carrot, 1 red cabbage"
4,IMG_1266.JPG,Greek Pasticcio & Roasted Vegetables,1. Put a large saucepan of well - salted water...,"2 red onions, 10g rosemary, 150g mushrooms, 1 ...","red onions, rosemary, mushrooms, garlic clov...","[, g, g, g, g, tsp, g, pack, tsps, g, , g, g, ]","[2, 10, 150, 50, 10, 1, 200, 1, 0.25, 175, 1, ...","[red onions, rosemary, mushrooms, walnuts, haz...","1 garlic clove, 25 g chopped toasted, 1 red pe..."


In [17]:
df.to_csv('cleaned2.csv')

## Top Ingredients 

In [8]:
dfi= df.explode('clean_ingredients')

In [9]:
dfi[['title', 'clean']].groupby('clean').count().sort_values('title', ascending=False).head(10)

TypeError: unhashable type: 'list'

# Get recipes based on ingredients

In [ ]:
x = input('What ingredients would you like to use (only comma separated)? ').split(',')

In [ ]:
mask = ~df.ingredients.isna()
for i in x:
    mask &= df.ingredients.str.contains(i, case=False) 
print(f'Found {len(df[mask])} recipes with those ingredients')
df[mask]

# Randomly give recipes

In [ ]:
n = int(input('How many recipes would you like?: '))

In [ ]:
q = int(input('For how many people are you cooking?: '))

In [69]:
sample = random.sample(range(1,len(df)),n)

In [70]:
df.loc[sample,'title']

75       Sticky Chicken Wings & Egg - fried Rice
191    Wild Garlic Potato Cakes & Carrot Chutney
87                    Lime & Chilli Tofu Fajitas
Name: title, dtype: object

In [71]:
s = ', '.join(df.loc[sample, 'ingredients'])

In [82]:
# s = '100g mushroom, 5 onion, 5 mushroom'

def agg_ingerdients(s, q): 
    obj = parse_ingredient(s)
    ingredients = obj.name.text.split(', ')
    whole = []
    d = defaultdict(list)
    for ing, a in zip(ingredients, obj.amount):
        d[ing].append(f'{float(a.quantity)*(3/2):.1f} {a.unit}')
    return d

d = agg_ingerdients(s,q)
out = ''
for k,v in d.items():
    v = ' + '.join(v)
    out += v + ' '+ k + ' \n'

print(out)

1.5 tbsp sesame seeds 
150.0 g white basmati rice 
225.0 g mushrooms 
300.0 g spring greens 
3.0  eggs 
1.5 tbsp tamari 
1.5 tbsp toasted sesame oil 
3.0 packs hoisin sauce 
3.0 packs sweet chilli sauce 
0.4 tsps + 0.4 tsps chilli flakes - add to 
750.0 g potatoes 
1.5  onion 
3.0 large carrots 
30.0 g raisins 
1.5 tbsp cider vinegar 
1.5 tbsp honey 
150.0 g wild garlic 
30.0 g plain flour 
37.5 g butter 
75.0 g watercress 
1.5 pack tortillas 
300.0 g sweet mixed peppers 
1.5  lime 
3.0 tsps chilli flakes - add 
1.5 bag coriander 
75.0 g mixed salad leaves 



# Get grocery list

In [10]:
df.head()

,original_filename,title,instructions,ingredients,clean_ingr,clean,clean_ingredients,whole,addon
0,IMG_1271.JPG,Greek Mushroom Ragù & Olive Oil Mash,"1. Peel, halve and finely slice the onion. Pee...","1 red onion, 2 garlic cloves, 200g portobello ...","red onion, garlic cloves, portobello mushroom...","[red onion, portobello mushrooms, green olives...","[red onion, portobello mushrooms, green olives...","[1 red onion, 200g portobello mushrooms, 60g g...","2 garlic cloves, 1 tin dark green lentils, 1 G..."
1,IMG_1270.JPG,Mushroom Risotto Stuffed Squash,1. Boil a kettle. Heat your oven to 190 ° C / ...,"1 leek, 10g rosemary, 5g dried mushrooms, 1 sp...","leek, rosemary, dried mushrooms, spaghetti s...","[leek, rosemary, mushrooms, bouillon powder, b...","[leek, rosemary, mushrooms, bouillon powder, b...","[1 leek, 10g rosemary, 1tbsp mushrooms, 200g b...","5 g dried, 1 spaghetti squash, 2 x 0.5 tbsp, 2..."
2,IMG_1268.JPG,Nasi Goreng with Peanuts & Sriracha,1. Boil a kettle.\n2. Using about a third of t...,"1/3 cucumber, 1 tbsp rice vinegar, 1 tbsp ligh...","cucumber, rice vinegar, light brown sugar, ...","[cucumber, rice vinegar, light brown sugar, br...","[cucumber, rice vinegar, light brown sugar, br...","[0.333 cucumber, 1tbsp rice vinegar, 1tbsp lig...","1 pointed cabbage, 1 large or 2 smaller carrot..."
3,IMG_1269.JPG,Teriyaki Mushroom Rice & Asian Slaw,1. Boil a kettle. Rinse the rice in a sieve. T...,"150g brown basmati rice, 1 red onion, 1 carrot...","brown basmati rice, red onion, carrot, red ca...","[brown basmati rice, sweet chilli sauce, chill...","[brown basmati rice, sweet chilli sauce, chill...","[150g brown basmati rice, 1pack sweet chilli s...","1 red onion, 1 carrot, 1 red cabbage"
4,IMG_1266.JPG,Greek Pasticcio & Roasted Vegetables,1. Put a large saucepan of well - salted water...,"2 red onions, 10g rosemary, 150g mushrooms, 1 ...","red onions, rosemary, mushrooms, garlic clov...","[red onions, rosemary, mushrooms, walnuts, haz...","[red onions, rosemary, mushrooms, walnuts, haz...","[2 red onions, 10g rosemary, 150g mushrooms, 5...","1 garlic clove, 25 g chopped toasted, 1 red pe..."


Categories: Rice, Pasta, Wraps, Couscus, Salad, Stirfry, 
Other: Stirfry, Salad,


In [11]:
df.to_csv('cleaned_recipes.csv')

# To Do
* aggregate ingredients
* in comment section there is an additional list of ingredients
* similar recipes: topic modelling on the instructions. i.e. sentence embeddings

# Done
* extract ingredients
* topic modelling on ingredients
* sample n number of recipes